In [2]:
import logging
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd
import pypsa
from pypsa.descriptors import get_switchable_as_dense as get_as_dense
from pypsa.linopf import (
    define_constraints,
    define_variables,
    get_var,
    ilopf,
    join_exprs,
    linexpr,
    network_lopf,
)

logger = logging.getLogger(__name__)

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\networkclustering.py:16: UserWarning:

The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 



In [ ]:
def add_RES_constraints(n, res_share):
    lgrouper = n.loads.bus.map(n.buses.country)
    ggrouper = n.generators.bus.map(n.buses.country)
    sgrouper = n.storage_units.bus.map(n.buses.country)
    cgrouper = n.links.bus0.map(n.buses.country)
    logger.warning(
        "The add_RES_constraints functionality is still work in progress. "
        "Unexpected results might be incurred, particularly if "
        "temporal clustering is applied or if an unexpected change of technologies "
        "is subject to the obtimisation."
    )
    load = (
        n.snapshot_weightings.generators
        @ n.loads_t.p_set.groupby(lgrouper, axis=1).sum()
    )
    rhs = res_share * load
    res_techs = [
        "solar",
        "onwind",
        "offwind-dc",
        "offwind-ac",
        "battery",
        "hydro",
        "ror",
    ]
    charger = ["H2 electrolysis", "battery charger"]
    discharger = ["H2 fuel cell", "battery discharger"]
    gens_i = n.generators.query("carrier in @res_techs").index
    stores_i = n.storage_units.query("carrier in @res_techs").index
    charger_i = n.links.query("carrier in @charger").index
    discharger_i = n.links.query("carrier in @discharger").index
    # Generators
    lhs_gen = (
        linexpr(
            (n.snapshot_weightings.generators, get_var(n, "Generator", "p")[gens_i].T)
        )
        .T.groupby(ggrouper, axis=1)
        .apply(join_exprs)
    )
    # StorageUnits
    lhs_dispatch = (
        (
            linexpr(
                (
                    n.snapshot_weightings.stores,
                    get_var(n, "StorageUnit", "p_dispatch")[stores_i].T,
                )
            )
            .T.groupby(sgrouper, axis=1)
            .apply(join_exprs)
        )
        .reindex(lhs_gen.index)
        .fillna("")
    )
    lhs_store = (
        (
            linexpr(
                (
                    -n.snapshot_weightings.stores,
                    get_var(n, "StorageUnit", "p_store")[stores_i].T,
                )
            )
            .T.groupby(sgrouper, axis=1)
            .apply(join_exprs)
        )
        .reindex(lhs_gen.index)
        .fillna("")
    )
    # Stores (or their resp. Link components)
    # Note that the variables "p0" and "p1" currently do not exist.
    # Thus, p0 and p1 must be derived from "p" (which exists), taking into account the link efficiency.
    lhs_charge = (
        (
            linexpr(
                (
                    -n.snapshot_weightings.stores,
                    get_var(n, "Link", "p")[charger_i].T,
                )
            )
            .T.groupby(cgrouper, axis=1)
            .apply(join_exprs)
        )
        .reindex(lhs_gen.index)
        .fillna("")
    )
    lhs_discharge = (
        (

            linexpr(
                (
                    n.snapshot_weightings.stores.apply(
                        lambda r: r * n.links.loc[discharger_i].efficiency
                    ),
                    get_var(n, "Link", "p")[discharger_i],
                )
            )
            .groupby(cgrouper, axis=1)
            .apply(join_exprs)
        )
        .reindex(lhs_gen.index)
        .fillna("")
    )
    # signs of resp. terms are coded in the linexpr.
    # todo: for links (lhs_charge and lhs_discharge), account for snapshot weightings
    lhs = lhs_gen + lhs_dispatch + lhs_store + lhs_charge + lhs_discharge
    define_constraints(n, lhs, "=", rhs, "RES share")
# Define RES shares and storage levels
res_shares = np.round(np.arange(0.10, 0.65, 0.05), 2)
storage_levels = list(range(0, 6))  # 0 to 5
base_path = os.getcwd()
base_filename = "elec_s_14_ec_lv1.0_RES{:.2f}-1H_dle.nc"
for res in res_shares:
    base_file = os.path.join(base_path, base_filename.format(res))
    
    if not os.path.exists(base_file):
        print(f"[!] Base file not found for RES {res:.2f}: {base_file}")
        continue

    print(f"\n=== RES {res:.2f} ===")
    for storage_level in storage_levels:
        print(f"  → Running with storage level {storage_level}")
        # Load base network
        n =pypsa.Network(base_file)
        for tech in ["battery", "H2"]:
            mask = n.storage_units.carrier == tech
            if storage_level == 0:
                # Disable storage
                n.storage_units.loc[mask, "p_nom_max"] = 0.0
                n.storage_units.loc[mask, "p_nom_min"] = 0.0
                n.storage_units.loc[mask, "p_nom"] = 0.0
                n.storage_units.loc[mask, "p_nom_extendable"] = False
            else:
                # Scale duration and capacity
                n.storage_units.loc[mask, "max_hours"] = (
                    n.storage_units.loc[mask, "max_hours"].astype(float) * storage_level
                )
                n.storage_units.loc[mask, "p_nom_max"] = (
                    n.storage_units.loc[mask, "p_nom_max"].astype(float) * storage_level
                )
                n.storage_units.loc[mask, "p_nom_extendable"] = True
        # Solve the network
        try:
            def make_extra_functionality(res_share):
                return lambda n, snapshots: add_RES_constraints(n, res_share)
            extra_func = make_extra_functionality(res)
            network_lopf(
                n,
                solver_name="cplex",
                solver_options = {
                    "threads": 4,
                    "workmem": 256,
                    "workdir": "./cplex_tmp",
                    "mip.tolerances.mipgap": 0.05,
                    "emphasis.memory": 1,                  # Prioritize memory conservation
                    "mip.limits.treememory": 512,         # Limit tree memory (MB)
                    "mip.strategy.file": 3,                # Write node files to disk (for large trees)
                    "mip.strategy.nodeselect": 1,          # Use best estimate node selection (less memory use)
                    "mip.strategy.variableselect": 4,      # Pseudocost branching (faster)
                    "mip.strategy.bbinterval": 100,        # Fewer backtracking intervals
                    "mip.limits.nodes": 50000  
                },
                extra_functionality=extra_func
            )
        except Exception as e:
            print(f"    [x] Optimization failed for RES {res} + Storage {storage_level}: {e}")
            continue
        # Save solved network
        output_file = f"{base_path}/elec_s_14_ec_lv1.0_RES{res:.2f}_STORAGE{storage_level}-1H_dle.nc"
        n.export_to_netcdf(output_file)
        print(f"    Saved to {output_file}")

[!] Base file not found for RES 0.10: c:\Users\bakht\Desktop\project\pypsa-earth\results\kz_default_2018_one_year1\networks\elec_s_14_ec_lv1.0_RES0.10-1H_dle.nc

=== RES 0.15 ===
  → Running with storage level 0


c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setti

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Memory                         1
CPXPARAM_MIP_Strategy_File                       3
CPXPARAM_MIP_Strategy_VariableSelect             4
CPXPARAM_MIP_Limits_Nodes                        50000
CPXPARAM_MIP_Strategy_BBInterval                 100
CPXPARAM_WorkMem                                 256
CPXPARAM_MIP_Tolerances_MIPGap                   0.050000000000000003
CPXPARAM_MIP_Limits_TreeMemory                   512
CPXPARAM_WorkDir                                 "./cplex_tmp"
Tried aggregator 2 times.
MIP Presolve eliminated 4075652 rows and 867422 columns.
Aggregator did 359160 substitutions.
Reduced MIP has 2432937 rows, 1804508 columns, and 9094390 nonzeros.
Reduced MIP has 201480 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 6.02 sec. (7401.05 ticks)
Found incumbent of value 2.5810991e+12 aft

INFO:pypsa.linopf:Optimization successful. Objective value: 2.94e+09
c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\linopf.py:1281: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\linopf.py:1281: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\linopf.py:1343: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\linopf.py:1415: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

INFO:pypsa.io:Exported network elec_s_14_ec_lv1.0_RES0.15_STORAGE0-1H_dle.nc has lines, generators, loads, stores, buses, carriers, links, global_constraints, storage_units


    Saved to c:\Users\bakht\Desktop\project\pypsa-earth\results\kz_default_2018_one_year1\networks/elec_s_14_ec_lv1.0_RES0.15_STORAGE0-1H_dle.nc
  → Running with storage level 1


...


Root node processing (before b&c):
  Real time             = 6389.12 sec. (3037909.34 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) = 6389.12 sec. (3037909.34 ticks)


CPLEX Error  1001: Out of memory.


    [x] Optimization failed for RES 0.15 + Storage 1: CPLEX Error  1001: Out of memory.
  → Running with storage level 2


c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.

c:\Users\bakht\anaconda3\envs\pypsa-earth\lib\site-packages\pypsa\components.py:318: FutureWarning:

Setti

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 4
CPXPARAM_Emphasis_Memory                         1
CPXPARAM_MIP_Strategy_File                       3
CPXPARAM_MIP_Strategy_VariableSelect             4
CPXPARAM_MIP_Limits_Nodes                        50000
CPXPARAM_MIP_Strategy_BBInterval                 100
CPXPARAM_WorkMem                                 256
CPXPARAM_MIP_Tolerances_MIPGap                   0.050000000000000003
CPXPARAM_MIP_Limits_TreeMemory                   512
CPXPARAM_WorkDir                                 "./cplex_tmp"
Tried aggregator 2 times.
MIP Presolve eliminated 3094532 rows and 131582 columns.
Aggregator did 359160 substitutions.
Reduced MIP has 3414057 rows, 2540376 columns, and 12283030 nonzeros.
Reduced MIP has 201480 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 10.76 sec. (7920.34 ticks)
